# Clasificación de 200 Especies de Aves - CUB-200-2011

Este notebook demuestra el uso de los tres modelos implementados:
1. CNN Base (desde cero)
2. ResNet50 (Transfer Learning)
3. EfficientNetB0 (Transfer Learning)

## Configuración Inicial

In [ ]:
import sys
import os

# Agregar el directorio raíz al path
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.models.base_cnn import create_base_cnn
from src.models.resnet50_model import create_resnet50_model, unfreeze_model
from src.models.efficientnet_model import create_efficientnet_model, unfreeze_efficientnet
from src.utils.data_utils import create_data_generators, get_test_generator
from src.utils.training_utils import get_callbacks
from src.utils.visualization import plot_training_history, compare_models, evaluate_model

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerías importadas correctamente")

## 1. Configuración de Parámetros

In [ ]:
# Parámetros
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 200
EPOCHS = 50

# Directorios
TRAIN_DIR = '../data/train'
VAL_DIR = '../data/val'
TEST_DIR = '../data/test'

print(f"Configuración:")
print(f"  - Tamaño de imagen: {IMG_SIZE}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Número de clases: {NUM_CLASSES}")
print(f"  - Épocas máximas: {EPOCHS}")

## 2. Carga de Datos con Data Augmentation

In [ ]:
# Crear generadores de datos
print("Creando generadores de datos...")
train_gen, val_gen = create_data_generators(
    TRAIN_DIR, VAL_DIR,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_gen = get_test_generator(TEST_DIR, img_size=IMG_SIZE, batch_size=BATCH_SIZE)

print(f"\n✓ Datos cargados:")
print(f"  - Entrenamiento: {train_gen.samples} imágenes")
print(f"  - Validación: {val_gen.samples} imágenes")
print(f"  - Prueba: {test_gen.samples} imágenes")
print(f"  - Clases: {len(train_gen.class_indices)}")

## 3. Visualización de Ejemplos del Dataset

In [ ]:
# Visualizar algunas imágenes del conjunto de entrenamiento
def plot_sample_images(generator, num_images=9):
    """
    Muestra imágenes de ejemplo del generador.
    """
    images, labels = next(generator)
    
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.ravel()
    
    class_names = {v: k for k, v in generator.class_indices.items()}
    
    for i in range(min(num_images, len(images))):
        axes[i].imshow(images[i])
        class_idx = np.argmax(labels[i])
        axes[i].set_title(f'Clase: {class_names[class_idx]}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

print("Ejemplos del conjunto de entrenamiento:")
plot_sample_images(train_gen)

## 4. Modelo 1: CNN Base (Desde Cero)

In [ ]:
print("="*70)
print("MODELO 1: CNN Base (desde cero)")
print("="*70)

# Crear modelo
cnn_model = create_base_cnn(input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES)

# Mostrar arquitectura
cnn_model.summary()

print("\n✓ Modelo CNN Base creado")

In [ ]:
# Entrenar (descomentar para ejecutar)
# callbacks = get_callbacks('base_cnn', patience=10)
# cnn_history = cnn_model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=EPOCHS,
#     callbacks=callbacks,
#     verbose=1
# )

print("Para entrenar este modelo, descomenta el código anterior")

## 5. Modelo 2: ResNet50 con Transfer Learning

In [ ]:
print("="*70)
print("MODELO 2: ResNet50 (Transfer Learning)")
print("="*70)

# Crear modelo
resnet_model = create_resnet50_model(
    input_shape=(*IMG_SIZE, 3),
    num_classes=NUM_CLASSES
)

print(f"\nTotal de parámetros: {resnet_model.count_params():,}")
print("\n✓ Modelo ResNet50 creado con Transfer Learning")

In [ ]:
# Fase 1: Entrenar capas superiores (descomentar para ejecutar)
# print("Fase 1: Entrenamiento de capas superiores...")
# callbacks = get_callbacks('resnet50_phase1', patience=10)
# resnet_history1 = resnet_model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=20,
#     callbacks=callbacks,
#     verbose=1
# )

# # Fase 2: Fine-tuning
# print("\nFase 2: Fine-tuning...")
# resnet_model = unfreeze_model(resnet_model, trainable_layers=50)
# callbacks = get_callbacks('resnet50_phase2', patience=10)
# resnet_history2 = resnet_model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=30,
#     callbacks=callbacks,
#     verbose=1
# )

print("Para entrenar este modelo, descomenta el código anterior")

## 6. Modelo 3: EfficientNetB0 con Transfer Learning

In [ ]:
print("="*70)
print("MODELO 3: EfficientNetB0 (Transfer Learning)")
print("="*70)

# Crear modelo
efficient_model = create_efficientnet_model(
    input_shape=(*IMG_SIZE, 3),
    num_classes=NUM_CLASSES
)

print(f"\nTotal de parámetros: {efficient_model.count_params():,}")
print("\n✓ Modelo EfficientNetB0 creado con Transfer Learning")

In [ ]:
# Fase 1: Entrenar capas superiores (descomentar para ejecutar)
# print("Fase 1: Entrenamiento de capas superiores...")
# callbacks = get_callbacks('efficientnet_phase1', patience=10)
# efficient_history1 = efficient_model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=20,
#     callbacks=callbacks,
#     verbose=1
# )

# # Fase 2: Fine-tuning
# print("\nFase 2: Fine-tuning...")
# efficient_model = unfreeze_efficientnet(efficient_model, unfreeze_from=100)
# callbacks = get_callbacks('efficientnet_phase2', patience=10)
# efficient_history2 = efficient_model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=30,
#     callbacks=callbacks,
#     verbose=1
# )

print("Para entrenar este modelo, descomenta el código anterior")

## 7. Comparación de Modelos

Esta sección compara los tres modelos entrenados.

In [ ]:
# Descomentar después de entrenar los modelos
# histories = [cnn_history, resnet_history, efficient_history]
# model_names = ['CNN Base', 'ResNet50', 'EfficientNetB0']
# compare_models(histories, model_names)

print("Comparación disponible después del entrenamiento")

## 8. Evaluación en Conjunto de Prueba

In [ ]:
# Descomentar después de entrenar los modelos
# print("Evaluación en conjunto de prueba:\n")

# # Evaluar CNN Base
# cnn_results = evaluate_model(cnn_model, test_gen)
# print(f"CNN Base - Accuracy: {cnn_results['test_accuracy']:.4f}, Loss: {cnn_results['test_loss']:.4f}")

# # Evaluar ResNet50
# resnet_results = evaluate_model(resnet_model, test_gen)
# print(f"ResNet50 - Accuracy: {resnet_results['test_accuracy']:.4f}, Loss: {resnet_results['test_loss']:.4f}")

# # Evaluar EfficientNetB0
# efficient_results = evaluate_model(efficient_model, test_gen)
# print(f"EfficientNetB0 - Accuracy: {efficient_results['test_accuracy']:.4f}, Loss: {efficient_results['test_loss']:.4f}")

print("Evaluación disponible después del entrenamiento")

## 9. Conclusiones

Este notebook demuestra:

1. **CNN Base**: Modelo construido desde cero, útil como baseline
2. **ResNet50**: Transfer Learning con arquitectura residual, excelente rendimiento
3. **EfficientNetB0**: Transfer Learning con arquitectura eficiente, mejor balance rendimiento/tamaño

### Resultados Esperados:

Los modelos de Transfer Learning (ResNet50 y EfficientNetB0) demuestran:
- Mayor precisión en validación y prueba
- Menor pérdida durante entrenamiento
- Convergencia más rápida
- Mejor generalización

### Técnicas Clave:

- **Data Augmentation**: Mejora la generalización
- **Early Stopping**: Previene sobreajuste
- **ReduceLROnPlateau**: Optimiza el aprendizaje
- **Fine-tuning**: Adapta modelos pre-entrenados al dominio específico